In [1]:
df = read.csv('../datasets/mediause.csv')
model = lm(formula = 'newspaper ~ age + gender', data = df)
model


Call:
lm(formula = "newspaper ~ age + gender", data = df)

Coefficients:
(Intercept)          age       gender  
   -0.08956      0.06762      0.17666  


In [2]:
gender = c(1,0)
age = c(20,40)
newdata = data.frame(age, gender)
predict(model, newdata)

1        2 
1.439508 2.615248

In [41]:
library(tidyverse)
library(caret)
library(rsample)
library(naivebayes)
set.seed(42)
df = na.omit(df %>% mutate(usesinternet=recode(internet, .default=TRUE, `0`=FALSE)))
df$usesinternet = as.factor(df$usesinternet)
print("How many people used online news at all?")
print(table(df$usesinternet))
print("Let's see how well we can predict this...")

split = initial_split(df, prop = .8)
traindata = training(split)
testdata  = testing(split)

X_train = select(traindata, c('age', 'gender', 'education'))
y_train = traindata$usesinternet
X_test = select(testdata, c('age', 'gender', 'education'))
y_test = testdata$usesinternet

myclassifier = train(x = X_train, y = y_train, method = "naive_bayes")
predictions = predict(myclassifier, newdata = X_test)
print(confusionMatrix(predictions, y_test))

print("Confusion matrix:")
confmat = table(testdata$usesinternet, predictions)
print(confmat)

print('Precision for predicting True internet users and non-internet-users, respecitively:')
precision = diag(confmat) / rowSums(confmat)
print(precision)


print('Recall for predicting True internet users and non-internet-users, respecitively:')
recall = (diag(confmat) / colSums(confmat))
print(recall)

[1] "How many people used online news at all?"

FALSE  TRUE 
  803  1262 
[1] "Let's see how well we can predict this..."
Confusion Matrix and Statistics

          Reference
Prediction FALSE TRUE
     FALSE    39   50
     TRUE     98  225
                                          
               Accuracy : 0.6408          
                 95% CI : (0.5924, 0.6872)
    No Information Rate : 0.6675          
    P-Value [Acc > NIR] : 0.8849694       
                                          
                  Kappa : 0.1128          
                                          
 Mcnemar's Test P-Value : 0.0001118       
                                          
            Sensitivity : 0.28467         
            Specificity : 0.81818         
         Pos Pred Value : 0.43820         
         Neg Pred Value : 0.69659         
             Prevalence : 0.33252         
         Detection Rate : 0.09466         
   Detection Prevalence : 0.21602         
      Balanced Accuracy : 0.

In [42]:
library(tidyverse)
library(caret)
library(rsample)
library(LogicReg)
set.seed(42)
df = na.omit(df %>% mutate(usesinternet=recode(internet, .default=TRUE, `0`=FALSE)))
df$usesinternet = as.factor(df$usesinternet)
print("How many people used online news at all?")
print(table(df$usesinternet))
print("Let's see how well we can predict this...")

split = initial_split(df, prop = .8)
traindata = training(split)
testdata  = testing(split)

X_train = select(traindata, c('age', 'gender', 'education'))
y_train = traindata$usesinternet
X_test = select(testdata, c('age', 'gender', 'education'))
y_test = testdata$usesinternet

myclassifier = train(x = X_train, y = y_train, method = "naive_bayes")
predictions = predict(myclassifier, newdata = X_test)
print(confusionMatrix(predictions, y_test))

print("Confusion matrix:")
confmat = table(testdata$usesinternet, predictions)
print(confmat)

print('Precision for predicting True internet users and non-internet-users, respecitively:')
precision = diag(confmat) / rowSums(confmat)
print(precision)


print('Recall for predicting True internet users and non-internet-users, respecitively:')
recall = (diag(confmat) / colSums(confmat))
print(recall)

[1] "How many people used online news at all?"

FALSE  TRUE 
  803  1262 
[1] "Let's see how well we can predict this..."
Confusion Matrix and Statistics

          Reference
Prediction FALSE TRUE
     FALSE    39   50
     TRUE     98  225
                                          
               Accuracy : 0.6408          
                 95% CI : (0.5924, 0.6872)
    No Information Rate : 0.6675          
    P-Value [Acc > NIR] : 0.8849694       
                                          
                  Kappa : 0.1128          
                                          
 Mcnemar's Test P-Value : 0.0001118       
                                          
            Sensitivity : 0.28467         
            Specificity : 0.81818         
         Pos Pred Value : 0.43820         
         Neg Pred Value : 0.69659         
             Prevalence : 0.33252         
         Detection Rate : 0.09466         
   Detection Prevalence : 0.21602         
      Balanced Accuracy : 0.

In [52]:
library(tidyverse)
library(caret)
library(rsample)
library(LiblineaR)
set.seed(42)
df = na.omit(df %>% mutate(usesinternet=recode(internet, .default=TRUE, `0`=FALSE)))
df$usesinternet = as.factor(df$usesinternet)
print("How many people used online news at all?")
print(table(df$usesinternet))
print("Let's see how well we can predict this...")

split = initial_split(df, prop = .8)
traindata = training(split)
testdata  = testing(split)

X_train = select(traindata, c('age', 'gender', 'education'))
y_train = traindata$usesinternet
X_test = select(testdata, c('age', 'gender', 'education'))
y_test = testdata$usesinternet

myclassifier = train(x = X_train, y = y_train, method = "svmLinear3")
predictions = predict(myclassifier, newdata = X_test)
print(confusionMatrix(predictions, y_test))

print("Confusion matrix:")
confmat = table(testdata$usesinternet, predictions)
print(confmat)

print('Precision for predicting True internet users and non-internet-users, respecitively:')
precision = diag(confmat) / rowSums(confmat)
print(precision)


print('Recall for predicting True internet users and non-internet-users, respecitively:')
recall = (diag(confmat) / colSums(confmat))
print(recall)

[1] "How many people used online news at all?"

FALSE  TRUE 
  803  1262 
[1] "Let's see how well we can predict this..."
Confusion Matrix and Statistics

          Reference
Prediction FALSE TRUE
     FALSE    38   56
     TRUE     99  219
                                         
               Accuracy : 0.6238         
                 95% CI : (0.575, 0.6707)
    No Information Rate : 0.6675         
    P-Value [Acc > NIR] : 0.9725792      
                                         
                  Kappa : 0.08           
                                         
 Mcnemar's Test P-Value : 0.0007421      
                                         
            Sensitivity : 0.27737        
            Specificity : 0.79636        
         Pos Pred Value : 0.40426        
         Neg Pred Value : 0.68868        
             Prevalence : 0.33252        
         Detection Rate : 0.09223        
   Detection Prevalence : 0.22816        
      Balanced Accuracy : 0.53687        
   

In [51]:
library(tidyverse)
library(caret)
library(rsample)
library(randomForest)
set.seed(42)
df = na.omit(df %>% mutate(usesinternet=recode(internet, .default=TRUE, `0`=FALSE)))
df$usesinternet = as.factor(df$usesinternet)
print("How many people used online news at all?")
print(table(df$usesinternet))
print("Let's see how well we can predict this...")

split = initial_split(df, prop = .8)
traindata = training(split)
testdata  = testing(split)

X_train = select(traindata, c('age', 'gender', 'education'))
y_train = traindata$usesinternet
X_test = select(testdata, c('age', 'gender', 'education'))
y_test = testdata$usesinternet

myclassifier = train(x = X_train, y = y_train, method = "rf")
predictions = predict(myclassifier, newdata = X_test)
print(confusionMatrix(predictions, y_test))

print("Confusion matrix:")
confmat = table(testdata$usesinternet, predictions)
print(confmat)

print('Precision for predicting True internet users and non-internet-users, respecitively:')
precision = diag(confmat) / rowSums(confmat)
print(precision)


print('Recall for predicting True internet users and non-internet-users, respecitively:')
recall = (diag(confmat) / colSums(confmat))
print(recall)

randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:dplyr’:

    combine

The following object is masked from ‘package:ggplot2’:

    margin



[1] "How many people used online news at all?"

FALSE  TRUE 
  803  1262 
[1] "Let's see how well we can predict this..."
note: only 2 unique complexity parameters in default grid. Truncating the grid to 2 .

Confusion Matrix and Statistics

          Reference
Prediction FALSE TRUE
     FALSE    52   81
     TRUE     85  194
                                         
               Accuracy : 0.5971         
                 95% CI : (0.548, 0.6448)
    No Information Rate : 0.6675         
    P-Value [Acc > NIR] : 0.9988         
                                         
                  Kappa : 0.0856         
                                         
 Mcnemar's Test P-Value : 0.8159         
                                         
            Sensitivity : 0.3796         
            Specificity : 0.7055         
         Pos Pred Value : 0.3910         
         Neg Pred Value : 0.6953         
             Prevalence : 0.3325         
         Detection Rate : 0.1262         
